In [6]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer

In [7]:
# read next fast speed
dataset = load_from_disk("../processed_datadir/wikitext-103-story-train")

In [3]:
print(dataset.features)
print(len(dataset))

{'text': Value(dtype='string', id=None)}
29525


In [ ]:
# for i in range(30):
#     print(len(dataset[i]['text']))

In [2]:
# from transformers import BertTokenizer
# import numpy as np

# # 初始化tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


In [3]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id


In [4]:
tokenizer.model_max_length = 2048

In [8]:


temp = tokenizer.batch_encode_plus(dataset[0:2]['text'],truncation=False)
# print(temp)                        # 这样搞直接把20000多的文本截取到只剩下2048
for k in temp.keys():
        if isinstance(temp[k],list):
                print(k, len(temp[k]))
        else:
                print(k, temp[k])
# print(temp['input_ids'])
# print(temp['attention_mask'])

Token indices sequence length is longer than the specified maximum sequence length for this model (4468 > 2048). Running this sequence through the model will result in indexing errors


input_ids 2
attention_mask 2


In [9]:
import multiprocessing

num_proc = multiprocessing.cpu_count() - 4
# num_proc = 1
print(num_proc)
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")

def group_texts(examples):
    # print(len(examples["text"]))
    # temp_string_list = []
    # for j in range(len(examples["text"])):
    #     text = examples["text"][j]
    #     temp_string_list.extend([text[i:i+tokenizer.model_max_length] for i in range(0, len(text), tokenizer.model_max_length)])
    # print("len of examples['text']:", len(examples["text"]))
    # print("len of examples['text'][0]:", len(examples["text"][0]))
    tokenized_inputs = tokenizer.batch_encode_plus(examples["text"], truncation = False)
    input_ids_list = []
    # token_type_ids_list = []
    attention_mask_list = []
    # print("len of tokenized_inputs['input_ids']:", len(tokenized_inputs["input_ids"]))
    # print("len of tokenized_inputs['input_ids'][0]:", len(tokenized_inputs["input_ids"][0]))
    # print("len of tokenized_inputs['attention_mask']:", len(tokenized_inputs["attention_mask"]))
    # print("len of tokenized_inputs['attention_mask'][0]:", len(tokenized_inputs["attention_mask"][0]))


    
    for input_ids, attention_mask in zip(tokenized_inputs["input_ids"], tokenized_inputs["attention_mask"]):
        # print(input_ids)
        # print(attention_mask)
        # 按照tokenizer.model_max_length的长度进行切断
        chunks_input_ids = [input_ids[i:i + tokenizer.model_max_length] for i in range(0, len(input_ids), tokenizer.model_max_length)]
        chunks_attention_mask = [attention_mask[i:i + tokenizer.model_max_length] for i in range(0, len(attention_mask), tokenizer.model_max_length)]

        # 对于不足tokenizer.model_max_length的部分进行补全
        chunks_input_ids = [chunk + [tokenizer.pad_token_id] * (tokenizer.model_max_length - len(chunk)) for chunk in chunks_input_ids]
        chunks_attention_mask = [chunk + [0] * (tokenizer.model_max_length - len(chunk)) for chunk in chunks_attention_mask]

        input_ids_list.extend(chunks_input_ids)
        attention_mask_list.extend(chunks_attention_mask)
    print("success")
    # return {"input_ids": input_ids_list, "token_type_ids": token_type_ids_list, "attention_mask": attention_mask_list}
    return {"input_ids": input_ids_list, "attention_mask": attention_mask_list}

# preprocess dataset
tokenized_datasets = dataset.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features

12
The max length for the tokenizer is: 2048


Map (num_proc=12):   0%|          | 0/29525 [00:00<?, ? examples/s]

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
successsuccess

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [10]:
print(len(tokenized_datasets)) # 这个理论上wikitext103数据集要在27万左右，如果是处理GPT只要7万

72224


In [ ]:
for i in range(0, len(tokenized_datasets)):
    # print(tokenized_datasets[i])
    for k in tokenized_datasets[i]:
        # print(k)
        # print(len(tokenized_datasets[i][k]))
        if len(tokenized_datasets[i][k]) != tokenizer.model_max_length:
            print("error")
            

In [11]:
# Split preprocessed dataset into train, validation, and test sets
splits = tokenized_datasets.train_test_split(test_size=0.03)
preprocessed_splits = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
    "test": load_from_disk("../processed_datadir/wikitext-103-story-test/")
})

In [12]:
preprocessed_splits.save_to_disk("../processed_datadir2/wikitext-103-story-gpt2-2048/")

Flattening the indices:   0%|          | 0/70057 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/70057 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/2167 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2167 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/62 [00:00<?, ? examples/s]

In [14]:
print(len(preprocessed_splits["train"]))

70057
